In [1]:
import pandas as pd
import json
from ast import literal_eval
from pandas.io.json import json_normalize
from datetime import datetime
import re
import requests
import numpy as np

In [2]:
#Pull website data and extract table
url = 'https://www.the-numbers.com/movies/franchises'

response = requests.get(url)

tables = pd.read_html(response.text)

f_all_data_df = tables[0]

In [3]:
#Keep relevant rows: Franchise name and Worldwide Box Office
franchise_df = f_all_data_df[["Franchise", "Worldwide Box Office"]]

In [4]:
#Remove currency formatting and drop franchises with no value or 0  
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace(',', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace('$', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].astype(float)

franchise_df.replace(0, np.nan, inplace=True)
franchise_df.dropna(inplace=True)

C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [5]:
#Export franchise data
#franchise_df.to_csv('franchise_df.csv', index=False)

In [6]:
#Read metadata CSV
csv_file = "new_movie_metadata.csv"

metadata_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape', low_memory=False)

In [7]:
#Keep relevant rows: original_title, collection, revenue. Drop rows with 0 value
franchise_single = metadata_df[['original_title', 'belongs_to_collection', 'revenue']]

franchise_single.astype({'revenue': 'float64'}).dtypes

franchise_single = franchise_single.query("revenue > 0")
franchise_single = franchise_single.reset_index(drop=True)
franchise_single.dropna(inplace=True)

In [8]:
#Export data for individual movies
#franchise_single.to_csv('metadata_franchise.csv', index=False)

In [9]:
#Create empty dataframe for final table
final_table = pd.DataFrame (columns = ['Franchise', 'Movie Title','Movie Revenue', 
                                       'Total Franchise Revenue', '% of Total Franchise Revenue'])

In [10]:
#Match if collection contains franchise name and input desired data into final table
for idx, fn in franchise_df.iterrows():
    search = fn["Franchise"]
    wwbo = fn["Worldwide Box Office"]
    title_match = franchise_single["belongs_to_collection"].str.contains(search)
    indexes = (title_match[title_match==True].index.values)
    
    revenue = franchise_single.loc[indexes, 'revenue'] 
    movie_title = franchise_single.loc[indexes, 'original_title'] 
    franchise_performance = (revenue / wwbo)*100

    final_data = {'Franchise': search, 'Movie Title': movie_title, 'Movie Revenue': revenue, 
                  'Total Franchise Revenue': wwbo, '% of Total Franchise Revenue': franchise_performance}
    final_table = final_table.append(pd.DataFrame(final_data, columns= ['Franchise', 'Movie Title', 
                                                                        'Movie Revenue', 'Total Franchise Revenue', 
                                                                        '% of Total Franchise Revenue']),ignore_index=True)

C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [11]:
#Clean final table
delete='[REC]'

clean_final = final_table[final_table.Franchise != delete]

#Export final table to CSV
clean_final.to_csv('Franchise_Performance_By_Movie.csv', index=False)

In [16]:
csv_file = "new_movie_metadata.csv"

ratings_data = "ratings.csv"


# df = pd.read_csv(csv_file,'unicode_escape')

movies_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')

ratings_df = pd.read_csv(ratings_data)

# csv_file.encode('utf-8').strip(csv_file)

In [ ]:
# df.head()

In [20]:
movies_df["genres"].head(1)

KeyError: 'genres'

In [ ]:
 # Step 2: Isolation of columns that are relevant
movie_genres = movies_df[['id', 'genres']]

In [ ]:
# Step 3: Iterate over each row from our isolated dataset
# to_string converts the movie_genre dataframe to a string. movie_genre is a variable prior.
parse_genre_df = pd.DataFrame()

for row_number, movie in movie_genres.iterrows():
    genre_list = []
#     print(type(movie_genres))
#     print(type(movie_genres.to_string()))
    for genre_counter, genre in enumerate(literal_eval(movie['genres'])):
#   print(movie['id'], genre.get('name'))
        genre_list.append(genre.get('name'))
            
        

    # Associate the parsed genres with the movie id
    parse_genre_df = parse_genre_df.append({"id": movie['id'],
                                            'genre': ', '.join(map(str, genre_list))},
                                           ignore_index=True,
                                           sort=False)
   

In [ ]:
# parse_genre_df.head()

In [17]:
prod_company = movies_df[["id", "production_companies"]]

In [19]:
parse_companies_df = pd.DataFrame()

for row_number, movie in prod_company.iterrows():
    company_list = []
    try:
        for company_counter, company in enumerate(literal_eval(movie['production_companies'])):
            company_list.append(company.get('name'))
    except ValueError:
        pass
        
    parse_companies_df = parse_companies_df.append({'id': movie['id'],
                                                   'production_companies': ', '.join(map(str, company_list))},
                                           ignore_index=True,
                                           sort=False)


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
# parse_companies_df.head()

In [ ]:
movies_df.drop(['genres', 'production_companies'], axis=1, inplace = True)

In [ ]:
movies_df.columns

In [ ]:
movies_df = movies_df.merge(parse_companies_df, on="id")

In [ ]:
movies_df.columns

In [ ]:
movies_df = movies_df.merge(parse_genre_df, on="id")

In [ ]:
movies_df. columns

In [ ]:
movies_df.head(5)


In [ ]:
movies_df.to_csv('new_movie_metadata.csv', index=False)

In [ ]:
movies_df = movies_df.drop(['adult', 'belongs_to_collection', 'genres', 'homepage', 'original_language',
                'original_title', 'overview', 'popularity', 'poster_path', 'production_countries',
                'runtime', 'spoken_languages', 'status', 'tagline', 'video', 'vote_count'], axis=1)

In [ ]:
movies_df.head()

In [ ]:
#renaming the columns
movies_df = movies_df.rename(columns={'budget': 'production_budget', 'id': 'movie_id', 'title': 'movie_name', 
                                      'vote_average': 'imdb_rating'})

In [ ]:
# Drop all blank movie ID's

movies_df = movies_df.dropna(subset=['movie_id'])

In [ ]:
# Removing the 0's before movie_id 

movies_df['movie_id'] = movies_df['movie_id'].astype(str).str.replace('\D+', '')

In [ ]:
# Setting movie_id to int

movies_df['movie_id'] = movies_df['movie_id'].apply(lambda x: int(x))

In [ ]:
#working on the RATING CSV here
ratings_df = ratings_df.groupby(['movieId'], as_index = False).mean()

In [ ]:
# Add rating to movies_df

movies_df['rating'] = movies_df.movie_id.map(ratings_df.set_index('movieId')['rating'].to_dict())

In [ ]:
movies_df.head()

In [ ]:
#Yearly Production Studio code here

In [ ]:
release_date = movies_df[['movie_id','release_date']].head()

In [ ]:
#a for loop to pick up dates in the excel that are messy.
year_list = []

for row_number, date in movies_df["release_date"].items():
    if isinstance(date, float) or len(date) <=3:
        year_list.append(None)
    else:
        try:
            date_year = datetime.strptime(date, "%m/%d/%Y")
            year_list.append(date_year.year)
        except ValueError:
            date_year = datetime.strptime(date, "%Y-%m-%d")
            year_list.append(date_year.year)

In [ ]:
#new column, who dis?
movies_df["release_year"] = year_list

In [ ]:
movies_df[["release_date","release_year"]]

In [ ]:
production = movies_df[["production_companies","revenue","production_budget","movie_name","release_year"]].copy()

In [ ]:
production = production.dropna(subset=['movie_name','production_companies'])

In [ ]:
top = production.sort_values(by = ["revenue"],ascending = False)

In [ ]:
#filtering production companies with Marvel in string
Marvel = production[production['production_companies'].str.contains('Marvel')]

In [ ]:
#looking at the most recent film in the table
sort_year = Marvel.sort_values(by = ["release_year"],ascending = False)

In [ ]:
#sorting by title since this table didn't have an index prior. dropped films that had 0 info
top_ten = sort_year.set_index('movie_name').drop(['Team Thor','Marvel Studios: Assembling a Universe',
                                             'Marvel One-Shot: All Hail the King',
                                             'Iron Man & Captain America: Heroes United',
                                             'Marvel: 75 Years, From Pulp to Pop!','Iron Man & Hulk: Heroes United',
                                             'Marvel One-Shot: Agent Carter','Marvel One-Shot: Item 47',
                                             'Thor: Tales of Asgard','Planet Hulk'])

In [ ]:
#reseting the index
top_ten.reset_index(level='movie_name')

In [ ]:
#convert budget to float since sum below wouldnt work as a object
top_ten["production_budget"] = top_ten["production_budget"].astype(float)

In [ ]:
revenue_sum = top_ten.groupby(['release_year'])['revenue'].sum()

budget_sum = top_ten.groupby(['release_year'])['production_budget'].sum()

In [ ]:
merge_inner = pd.merge(left=revenue_sum, right=budget_sum, left_on='release_year', right_on='release_year')

In [ ]:
#final table with total revenue and budget per year
merge_inner.sort_values(by = ["release_year"],ascending = False)